In [1]:
import os
import time
import pandas as pd 
import requests
import urllib3
import json
import sys
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta

pd.options.display.max_colwidth = 1000
pd.set_option('display.max_columns', None)

### Files Yara Rules

In [2]:
file_reddit_dir = r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\SocialNetworksPosts\Reddit\RedditPosts.csv'

In [3]:
df_reddit = pd.read_csv(file_reddit_dir, low_memory = False)
df_reddit = df_reddit.drop(columns=['Unnamed: 0', 'json'])
df_reddit

,subreddit,username,date,title,text
0,netsec,t3_18i5ups,2023-12-14 11:36:39,Nim implementation of Old Dll Unlinking Technique,NaN
1,cybersecurity,t3_18i5v5x,2023-12-14 11:37:34,How to prevent employees from using unauthorized web tools?,"I work as an IT manager for a business. Some time ago, a group of my colleagues decided to use a web productivity tool, created accounts using their work emails, and started introducing data.\n\nI had no knowledge they had been using such a tool at all. I did not even know it existed.\n\nToday, I received an email from my boss asking why an ex-employee still had access to the data in such a web platform, as their account hadn't been deactivated. Needless to say, I don't have access to revoke their account or anything.\n\nHow can I prevent them from registering on another unauthorized webpage?\n\nI feel like a firewall blacklist would be useless since I would have to be constantly researching and updating it with new tools, and I can't make a whitelist since my company uses a ton of different pages for intense web research.\n\nAny help or advice would be appreciated.\n\n&#x200B;"
2,MacOS,t3_18i5kyd,2023-12-14 11:17:47,Teams call pop up window blocked by reminders/calendar entries on top right corner,"Hi\n\nDo you know this issue that when I have reminders or calendar alarms set to stick to the top right corner, that they then block the notification window for incoming Teams calls.\n\nDoes someone know if there is a setting for having the teams call pop up in on another location on the screen or knows another work around?"
3,MacOS,t3_18i5koq,2023-12-14 11:17:16,Disable middle mouse button drag to other desktop,"So I have been using windows for a while and now I am stumped. The middle mouse button of my logi mouse has the following functionailty: On drage change desktop, on click show all windows. That's nice and all, but I can't use fusion 360 because the dragging functionality is the middle mouse button but that just changes the desktop. How can I disable that?\n\n&#x200B;\n\nSo I found this, but there is no button selected. If I select the secondary mouse button for example, it will open mission control on both right click and middle mouse button:\n\nhttps://preview.redd.it/bmqnrpw0m86c1.png?width=476&format=png&auto=webp&s=97ae662fcca43bf146537393fc6a13705a0a9351\n\n&#x200B;"
4,windows,t3_18i5gqv,2023-12-14 11:09:52,Playing 4k videos on 2k display,Hello guys does playing 4k content on a 2k display has any benefit? For example while playing YouTube videos I set 1440p resolution on my 2k laptop and 2k smartphone despite having a option of selecting 2160p option. So i wanted to ask if i get better video quality if i select 2160p option while playing on a 2k display?
...,...,...,...,...,...
387066,pcmasterrace,t3_1006rmg,2023-01-01 00:41:37,Build Almost Done,Is this a good build? Any recommendations?\n\nBUILD - https://pcpartpicker.com/list/FL6hwc
387067,pcmasterrace,t3_1006qur,2023-01-01 00:40:30,"problem arming pc, I have a Mother a320m-k, ryzen 4650g, and doesnt work the video, the motherboard have Orange leds. Its my first time arming",NaN
387068,pcmasterrace,t3_1006pzy,2023-01-01 00:39:17,PCMR Truckers,"Those that bring their rigs with them in the truck: \n\nHow does the computer fare? \nIs there any orientation it works better with when you go through the areas with poor maintenance? \nAnd lastly: if you liquid cool, have you noticed any issues over time?\n\nI'm building a new rig here soon and am curious if anything is to note so I don't make a large $$$$ mistake"
387069,pcmasterrace,t3_1006orx,2023-01-01 00:37:29,"New case for Christmas, also spray painted my GPU to match","Got the Hyte y60 as a gift, what a great case to build in. Running a 5600x with a 3080. The tree probably blocks some airflow but I'll take that tradeoff."


In [4]:
df_reddit['text'] = df_reddit['text'].fillna(' ')
df_reddit['title'] = df_reddit['title'].fillna(' ')

df_reddit

,subreddit,username,date,title,text
0,netsec,t3_18i5ups,2023-12-14 11:36:39,Nim implementation of Old Dll Unlinking Technique,
1,cybersecurity,t3_18i5v5x,2023-12-14 11:37:34,How to prevent employees from using unauthorized web tools?,"I work as an IT manager for a business. Some time ago, a group of my colleagues decided to use a web productivity tool, created accounts using their work emails, and started introducing data.\n\nI had no knowledge they had been using such a tool at all. I did not even know it existed.\n\nToday, I received an email from my boss asking why an ex-employee still had access to the data in such a web platform, as their account hadn't been deactivated. Needless to say, I don't have access to revoke their account or anything.\n\nHow can I prevent them from registering on another unauthorized webpage?\n\nI feel like a firewall blacklist would be useless since I would have to be constantly researching and updating it with new tools, and I can't make a whitelist since my company uses a ton of different pages for intense web research.\n\nAny help or advice would be appreciated.\n\n&#x200B;"
2,MacOS,t3_18i5kyd,2023-12-14 11:17:47,Teams call pop up window blocked by reminders/calendar entries on top right corner,"Hi\n\nDo you know this issue that when I have reminders or calendar alarms set to stick to the top right corner, that they then block the notification window for incoming Teams calls.\n\nDoes someone know if there is a setting for having the teams call pop up in on another location on the screen or knows another work around?"
3,MacOS,t3_18i5koq,2023-12-14 11:17:16,Disable middle mouse button drag to other desktop,"So I have been using windows for a while and now I am stumped. The middle mouse button of my logi mouse has the following functionailty: On drage change desktop, on click show all windows. That's nice and all, but I can't use fusion 360 because the dragging functionality is the middle mouse button but that just changes the desktop. How can I disable that?\n\n&#x200B;\n\nSo I found this, but there is no button selected. If I select the secondary mouse button for example, it will open mission control on both right click and middle mouse button:\n\nhttps://preview.redd.it/bmqnrpw0m86c1.png?width=476&format=png&auto=webp&s=97ae662fcca43bf146537393fc6a13705a0a9351\n\n&#x200B;"
4,windows,t3_18i5gqv,2023-12-14 11:09:52,Playing 4k videos on 2k display,Hello guys does playing 4k content on a 2k display has any benefit? For example while playing YouTube videos I set 1440p resolution on my 2k laptop and 2k smartphone despite having a option of selecting 2160p option. So i wanted to ask if i get better video quality if i select 2160p option while playing on a 2k display?
...,...,...,...,...,...
387066,pcmasterrace,t3_1006rmg,2023-01-01 00:41:37,Build Almost Done,Is this a good build? Any recommendations?\n\nBUILD - https://pcpartpicker.com/list/FL6hwc
387067,pcmasterrace,t3_1006qur,2023-01-01 00:40:30,"problem arming pc, I have a Mother a320m-k, ryzen 4650g, and doesnt work the video, the motherboard have Orange leds. Its my first time arming",
387068,pcmasterrace,t3_1006pzy,2023-01-01 00:39:17,PCMR Truckers,"Those that bring their rigs with them in the truck: \n\nHow does the computer fare? \nIs there any orientation it works better with when you go through the areas with poor maintenance? \nAnd lastly: if you liquid cool, have you noticed any issues over time?\n\nI'm building a new rig here soon and am curious if anything is to note so I don't make a large $$$$ mistake"
387069,pcmasterrace,t3_1006orx,2023-01-01 00:37:29,"New case for Christmas, also spray painted my GPU to match","Got the Hyte y60 as a gift, what a great case to build in. Running a 5600x with a 3080. The tree probably blocks some airflow but I'll take that tradeoff."


In [5]:
df_reddit['date'] = pd.to_datetime(df_reddit['date']).dt.strftime('%Y-%m-%d')
df_reddit

,subreddit,username,date,title,text
0,netsec,t3_18i5ups,2023-12-14,Nim implementation of Old Dll Unlinking Technique,
1,cybersecurity,t3_18i5v5x,2023-12-14,How to prevent employees from using unauthorized web tools?,"I work as an IT manager for a business. Some time ago, a group of my colleagues decided to use a web productivity tool, created accounts using their work emails, and started introducing data.\n\nI had no knowledge they had been using such a tool at all. I did not even know it existed.\n\nToday, I received an email from my boss asking why an ex-employee still had access to the data in such a web platform, as their account hadn't been deactivated. Needless to say, I don't have access to revoke their account or anything.\n\nHow can I prevent them from registering on another unauthorized webpage?\n\nI feel like a firewall blacklist would be useless since I would have to be constantly researching and updating it with new tools, and I can't make a whitelist since my company uses a ton of different pages for intense web research.\n\nAny help or advice would be appreciated.\n\n&#x200B;"
2,MacOS,t3_18i5kyd,2023-12-14,Teams call pop up window blocked by reminders/calendar entries on top right corner,"Hi\n\nDo you know this issue that when I have reminders or calendar alarms set to stick to the top right corner, that they then block the notification window for incoming Teams calls.\n\nDoes someone know if there is a setting for having the teams call pop up in on another location on the screen or knows another work around?"
3,MacOS,t3_18i5koq,2023-12-14,Disable middle mouse button drag to other desktop,"So I have been using windows for a while and now I am stumped. The middle mouse button of my logi mouse has the following functionailty: On drage change desktop, on click show all windows. That's nice and all, but I can't use fusion 360 because the dragging functionality is the middle mouse button but that just changes the desktop. How can I disable that?\n\n&#x200B;\n\nSo I found this, but there is no button selected. If I select the secondary mouse button for example, it will open mission control on both right click and middle mouse button:\n\nhttps://preview.redd.it/bmqnrpw0m86c1.png?width=476&format=png&auto=webp&s=97ae662fcca43bf146537393fc6a13705a0a9351\n\n&#x200B;"
4,windows,t3_18i5gqv,2023-12-14,Playing 4k videos on 2k display,Hello guys does playing 4k content on a 2k display has any benefit? For example while playing YouTube videos I set 1440p resolution on my 2k laptop and 2k smartphone despite having a option of selecting 2160p option. So i wanted to ask if i get better video quality if i select 2160p option while playing on a 2k display?
...,...,...,...,...,...
387066,pcmasterrace,t3_1006rmg,2023-01-01,Build Almost Done,Is this a good build? Any recommendations?\n\nBUILD - https://pcpartpicker.com/list/FL6hwc
387067,pcmasterrace,t3_1006qur,2023-01-01,"problem arming pc, I have a Mother a320m-k, ryzen 4650g, and doesnt work the video, the motherboard have Orange leds. Its my first time arming",
387068,pcmasterrace,t3_1006pzy,2023-01-01,PCMR Truckers,"Those that bring their rigs with them in the truck: \n\nHow does the computer fare? \nIs there any orientation it works better with when you go through the areas with poor maintenance? \nAnd lastly: if you liquid cool, have you noticed any issues over time?\n\nI'm building a new rig here soon and am curious if anything is to note so I don't make a large $$$$ mistake"
387069,pcmasterrace,t3_1006orx,2023-01-01,"New case for Christmas, also spray painted my GPU to match","Got the Hyte y60 as a gift, what a great case to build in. Running a 5600x with a 3080. The tree probably blocks some airflow but I'll take that tradeoff."


### NLP sul Behaviours in ogni file

#### Example of NLP

In [6]:
!python -m spacy download it_core_news_sm


     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     --------------------------------------- 0.0/13.0 MB 222.6 kB/s eta 0:00:59
     --------------------------------------- 0.0/13.0 MB 330.3 kB/s eta 0:00:40
     --------------------------------------- 0.1/13.0 MB 950.9 kB/s eta 0:00:14
      --------------------------------------- 0.2/13.0 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.4/13.0 MB 1.5 MB/s eta 0:00:09
     - -------------------------------------- 0.5/13.0 MB 1.9 MB/s eta 0:00:07
     - -------------------------------------- 0.6/13.0 MB 2.0 MB/s eta 0:00:07
     -- ------------------------------------- 0.7/13.0 MB 1.9 MB/s eta 0:00:07
     -- ------------------------------------- 0.8/13.0 MB 1.9 MB/s eta 0:00:07
     -- ------------------------------------- 0.9/13.0 MB 1.9 MB/s eta 0:00:07
     -- ------------------------------------- 0.9/13.0 MB 1.9 M

In [7]:
import spacy

nlp = spacy.load("it_core_news_sm")

frase = "Il gatto mangia il pesce."

doc = nlp(frase)

# Estrai sostantivi e verbi dalla frase
parole_di_interesse = [token.text for token in doc if token.pos_ in ['NOUN', 'VERB']]

# Stampa i risultati
print("Parole di interesse:", parole_di_interesse)

Parole di interesse: ['gatto', 'mangia', 'pesce']


#### Get Words from descriptions

In [8]:
pip install spacy


Note: you may need to restart the kernel to use updated packages.


In [9]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.1/12.8 MB 469.7 kB/s eta 0:00:28
     --------------------------------------- 0.2/12.8 MB 833.5 kB/s eta 0:00:16
      --------------------------------------- 0.3/12.8 MB 1.4 MB/s eta 0:00:10
     - -------------------------------------- 0.3/12.8 MB 1.4 MB/s eta 0:00:09
     - -------------------------------------- 0.4/12.8 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.5/12.8 MB 1.4 MB/s eta 0:00:09
     - -------------------------------------- 0.6/12.8 MB 1.5 MB/s eta 0:00:09
     -- ------------------------------------- 0.7/12.8 MB 1.5 MB/s eta 0:00:09
     -- ------------------------------------- 0.8/12.8 MB 1.6 MB/s eta 0:00:08
     -- ------------------------------------- 0.9/12.8 MB 1.7 MB/s eta 

In [10]:
selected_rows = df_reddit.loc[(pd.to_datetime(df_reddit['date']).dt.month == 10) & (pd.to_datetime(df_reddit['date']).dt.year == 2023) ].reset_index()
selected_rows

,index,subreddit,username,date,title,text
0,60955,selfhosted,t3_17kxson,2023-10-31,Nginx Web UI - How to PHP?,"How to install PHP in [https://nginxui.com](https://nginxui.com)? \nI tried to \n\n\ntry\_files $uri =404;\n\nfastcgi\_pass unix:/var/run/php/php8.1-fpm.sock;\n\nfastcgi\_index index.php;\n\ninclude fastcgi.conf;\n\n \n\n\nbut error occures... \n \n\nNginx Configuration Parse Error\n\n2023/10/31 23:53:32 \[emerg\] 7645#7645: open() ""/etc/nginx/fastcgi.conf"" failed (2: No such file or directory) in /etc/nginx/sites-enabled/axis-group:10 nginx: \[emerg\] open() ""/etc/nginx/fastcgi.conf"" failed (2: No such file or directory) in /etc/nginx/sites-enabled/axis-group:10 nginx: configuration file /etc/nginx/nginx.conf test failed"
1,60959,HomeNetworking,t3_17kxosp,2023-10-31,High efficient and low idle power PoE++ Adapter (802.3bt/60W) searched,"Hi,\n\ndoes someone of you know a high efficient and low idle power using PoE++ (802.3bt/60W) adapter? \n\nI already tried the TP-Link TL-POE170S, but without having any LAN cables connected and being idle the device already draws around 2 watts. \n\nUnfortunately I live in the country with the highest energy prices of the world and therefore have to try to save as many watts as possible.\n\nI have good experience (idle usage almost 0 watt) with the PoE+ adapter from Ubiquiti, but it doesn't look like they also have PoE++ adapters."
2,60969,learnpython,t3_17kxrk5,2023-10-31,supersets confusion,"On HackerRank, qst: Check Strict Superset \nI provided the following code but it's not passing 2 of the 6 test cases, I'm not sure why \n\n`a = set(map(int, input().split()))`\r \n`n = int(input())`\r \n`res = []`\r \n`for _ in range(n):`\r \n`s = set(map(int, input().split()))`\r \n`if len(s) == len(a):`\r \n`res.append(False)`\r \n`else:`\r \n`res.append(s.issuperset(a))`\r \n \n`print(all(res))`\n\nThe question is as follows : \n\nYou are given a set A and other sets.\r \nYour job is to find whether set A is a strict superset of each of the N sets.\r \n\r \nPrint True, if A is a strict superset of each of the N sets. Otherwise, print False.\r \n\r \nA strict superset has at least one element that does not exist in its subset.\r\r \n\r \nInput Format\r \n\r \nThe first line contains the space separated elements of set\r A.\r \nThe second line contains integer n, the number of other sets.\r \nThe next n lines contains the space separated elements of the other..."
3,60978,AndroidQuestions,t3_17kxt7g,2023-10-31,"""Can't load android system"" error","looked all over the internet and found fixes only for the pixel, and not the phone i actually needed it for, the Cricket ovation 2"
4,60979,AndroidQuestions,t3_17kxmpr,2023-10-31,Sim issue,"I have two sims, only one works properly on two of my phones I tried all the four slots and the sim works on all of them \n\nBut the problem comes with the second sim, it kept on getting disabled on both of my phones and I had an issue with sim swapping in the past for which I got a new sim of the second one\n\nSo I decided to lock my sim and it ended up getting blocked to many retries I think it may have been changed\n\nSo I get it unlocked and they say its an issue with my phone so I put it in my other one still the same says that sim is unusable \n\nBut I somehow decided to try my friends phone and it somehow worked with calls and all\n\nSo I wondered why only one sim worked while the other didn't and it had worked in a phone \nI also had an issue with malware on my phone could it be possible for it to do all this?"
...,...,...,...,...,...,...
44475,105488,realtech,t3_16wj7lz,2023-10-01,Apple briefly considered buying Bing from Microsoft to replace Google search on iPhone,
44476,105489,Xiaomi,t3_16wk5k7,2023-10-01,Uninstall bloatware/System apks without root or unlocking bootloader,https://youtu.be/3_rqhoMpr_Y?si=gCjYK-3VSuKAfJMJ\n\nA simple guide to uninstall any system app without root. :)
44477,105490,linuxquestions,t3_16wkd6g,2023-10-01,How do I uninsta

In [11]:
from tqdm import tqdm

def create_words_df(df):
    nlp = spacy.load("en_core_web_sm")
    combined_texts = df['title'] + ' . ' + df['text'] 
    
    data_list = []
    
    for i, text in tqdm(enumerate(combined_texts), total=len(combined_texts), desc="Processing documents"):
        doc = nlp(text)
        words_interesting = [token.text for token in doc if token.pos_ in ['NOUN', 'VERB']]
        
        for word in words_interesting:
            word_type = 'noun' if nlp(word)[0].pos_ == 'NOUN' else 'verb'
            data_list.append({'word': word, 'type': word_type, 'index': i})
            
    dataframe = pd.DataFrame(data_list).reset_index()
    
    return dataframe


In [12]:
words_df = create_words_df(selected_rows)

Processing documents: 100%|██████████| 44480/44480 [2:16:27<00:00,  5.43it/s]  


In [13]:
words_df

,level_0,word,type,index
0,0,Web,noun,0
1,1,install,verb,0
2,2,tried,verb,0
3,3,try\_files,noun,0
4,4,=,verb,0
...,...,...,...,...
1539877,1539877,roof,noun,44479
1539878,1539878,speeds,noun,44479
1539879,1539879,fan,noun,44479
1539880,1539880,going,verb,44479


In [14]:
words_df.to_csv(r'C:\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\SocialNetworksPosts\Reddit\PostsAnalysis.csv')

In [15]:
merged_words_df = pd.merge(selected_rows, words_df, left_index=True, right_on='index')
merged_words_df = merged_words_df.drop(columns=['index'])

merged_words_df.head()


,index_x,subreddit,username,date,title,text,level_0,word,type,index_y
0,60955,selfhosted,t3_17kxson,2023-10-31,Nginx Web UI - How to PHP?,"How to install PHP in [https://nginxui.com](https://nginxui.com)? \nI tried to \n\n\ntry\_files $uri =404;\n\nfastcgi\_pass unix:/var/run/php/php8.1-fpm.sock;\n\nfastcgi\_index index.php;\n\ninclude fastcgi.conf;\n\n \n\n\nbut error occures... \n \n\nNginx Configuration Parse Error\n\n2023/10/31 23:53:32 \[emerg\] 7645#7645: open() ""/etc/nginx/fastcgi.conf"" failed (2: No such file or directory) in /etc/nginx/sites-enabled/axis-group:10 nginx: \[emerg\] open() ""/etc/nginx/fastcgi.conf"" failed (2: No such file or directory) in /etc/nginx/sites-enabled/axis-group:10 nginx: configuration file /etc/nginx/nginx.conf test failed",0,Web,noun,0
1,60955,selfhosted,t3_17kxson,2023-10-31,Nginx Web UI - How to PHP?,"How to install PHP in [https://nginxui.com](https://nginxui.com)? \nI tried to \n\n\ntry\_files $uri =404;\n\nfastcgi\_pass unix:/var/run/php/php8.1-fpm.sock;\n\nfastcgi\_index index.php;\n\ninclude fastcgi.conf;\n\n \n\n\nbut error occures... \n \n\nNginx Configuration Parse Error\n\n2023/10/31 23:53:32 \[emerg\] 7645#7645: open() ""/etc/nginx/fastcgi.conf"" failed (2: No such file or directory) in /etc/nginx/sites-enabled/axis-group:10 nginx: \[emerg\] open() ""/etc/nginx/fastcgi.conf"" failed (2: No such file or directory) in /etc/nginx/sites-enabled/axis-group:10 nginx: configuration file /etc/nginx/nginx.conf test failed",1,install,verb,0
2,60955,selfhosted,t3_17kxson,2023-10-31,Nginx Web UI - How to PHP?,"How to install PHP in [https://nginxui.com](https://nginxui.com)? \nI tried to \n\n\ntry\_files $uri =404;\n\nfastcgi\_pass unix:/var/run/php/php8.1-fpm.sock;\n\nfastcgi\_index index.php;\n\ninclude fastcgi.conf;\n\n \n\n\nbut error occures... \n \n\nNginx Configuration Parse Error\n\n2023/10/31 23:53:32 \[emerg\] 7645#7645: open() ""/etc/nginx/fastcgi.conf"" failed (2: No such file or directory) in /etc/nginx/sites-enabled/axis-group:10 nginx: \[emerg\] open() ""/etc/nginx/fastcgi.conf"" failed (2: No such file or directory) in /etc/nginx/sites-enabled/axis-group:10 nginx: configuration file /etc/nginx/nginx.conf test failed",2,tried,verb,0
3,60955,selfhosted,t3_17kxson,2023-10-31,Nginx Web UI - How to PHP?,"How to install PHP in [https://nginxui.com](https://nginxui.com)? \nI tried to \n\n\ntry\_files $uri =404;\n\nfastcgi\_pass unix:/var/run/php/php8.1-fpm.sock;\n\nfastcgi\_index index.php;\n\ninclude fastcgi.conf;\n\n \n\n\nbut error occures... \n \n\nNginx Configuration Parse Error\n\n2023/10/31 23:53:32 \[emerg\] 7645#7645: open() ""/etc/nginx/fastcgi.conf"" failed (2: No such file or directory) in /etc/nginx/sites-enabled/axis-group:10 nginx: \[emerg\] open() ""/etc/nginx/fastcgi.conf"" failed (2: No such file or directory) in /etc/nginx/sites-enabled/axis-group:10 nginx: configuration file /etc/nginx/nginx.conf test failed",3,try\_files,noun,0
4,60955,selfhosted,t3_17kxson,2023-10-31,Nginx Web UI - How to PHP?,"How to install PHP in [https://nginxui.com](https://nginxui.com)? \nI tried to \n\n\ntry\_files $uri =404;\n\nfastcgi\_pass unix:/var/run/php/php8.1-fpm.sock;\n\nfastcgi\_index index.php;\n\ninclude fastcgi.conf;\n\n \n\n\nbut error occures... \n \n\nNginx Configuration Parse Error\n\n2023/10/31 23:53:32 \[emerg\] 7645#7645: open() ""/etc/nginx/fastcgi.conf"" failed (2: No such file or directory) in /etc/nginx/sites-enabled/axis-group:10 nginx: \[emerg\] open() ""/etc/nginx/fastcgi.conf"" failed (2: No such file or directory) in /etc/nginx/sites-enabled/axis-group:10 nginx: configuration file /etc/nginx/nginx.conf test failed",4,=,verb,0


In [16]:
merged_words_df.to_csv(r'C:\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\SocialNetworksPosts\Reddit\PostsAnalysis.csv')